# Inter-Boundary Distance Measurement
TJ Kim

11.9.21

### Summary:
- Given multiple models performing identical task and a test set, measure the inter-boundary distance to measure transferability between models.
- Legitimate direction - any point x and closest data point in test set that changes the class
- Adversarial direction - any point x and smallest amount of perturbation that changes class
- Random direction - any point x and random perturbation drawn uniformly to misclassify

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


### Import Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

In [3]:
# Boundary Transferer Imports
# Import Custom Made Victim
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Attack_Metrics import *
import pandas as pd
from transfer_attacks.Transferer import *


### Boundary Transferer Class
- Used to calcualte direction of all 3 types
- Based off of (not subclass of) the Transferer class

In [4]:
class Boundary_Transferer(): 
    """
    - Load all the datasets but separate them
    - Intermediate values of featues after 2 convolution layers
    """
    
    def __init__(self, models_list, dataloader):
        
        self.models_list = models_list
        self.dataloader = dataloader
        
        self.advNN_idx = None
        self.advNN = None # Dictionary to hold adversary 
        self.atk_order = None
        
        self.base_nn_idx = None
        self.victim_idx = None
        
        self.fixed_point = None # {"idx","x","y"}
        self.comparison_set = None # For legitimate direction
        self.comparison_x = None
        self.comparison_y = None
        
        # Target that is the closest point in legit used in adv targeted attack
        self.y_comparison = None
        
        # Put single model from model_list into adversarial NN (can perform attacks)
        adv_nn = None
        self.atk_params = IFSGM_Params()
        
    def set_adv_NN(self, idx):
        
        self.advNN = copy.deepcopy(Adv_NN(self.models_list[idx], self.dataloader))
        
    def set_ensemble_adv_NN(self, client_idx):
        
        self.advNN_idx = client_idx # List
        self.advNN = {} # Dict of NN
        
        for i in client_idx:
            self.advNN[i] = copy.deepcopy(Adv_NN(self.models_list[i], self.dataloader))
        
        return
        
    def select_data_point(self, explore_set_size = 100):
        """
        Select a single data point to use as comparison of different boundary types
        Change - all members of the system must classify this point the same 
        """
        self.fixed_point = {}
        found_point_flag = False
        
        while not found_point_flag:
            x, y = self.dataloader.load_batch(explore_set_size)
            correct_idx = {}

            # Analyze Dataset and find point that is classified correctly
            for nidx in range(len(self.models_list)):
                preds = torch.argmax(self.models_list[nidx](x),axis=1)
                correct_idx[nidx] = torch.where(preds == y)[0]

            temp_idx = correct_idx[0]

            for nidx in range(1,len(self.models_list)):
                comp_idx = correct_idx[nidx]
                indices = torch.zeros_like(temp_idx,dtype=torch.bool,device='cuda')
                for elem in comp_idx:
                    indices = indices | (temp_idx==elem)
                temp_idx = temp_idx[indices]

            # Select a point within remaining at random
            # print(temp_idx.numel())
            if not temp_idx.numel():
                continue
            else:
                chosen_idx = temp_idx[np.random.randint(temp_idx.shape[0],size=1)[0]]
                self.fixed_point["x"] = x[chosen_idx]
                self.fixed_point["y"] = y[chosen_idx]
                found_point_flag = True
            
        return 
    
    def select_comparison_set(self,batch_size):
        """
        Select multiple datapoints to use to compare 
        """
        
        xs, ys = self.dataloader.load_batch(batch_size)
        
        self.comparison_set = {}
        
        self.comparison_set["x"] = xs
        self.comparison_set["y"] = ys
        
        # Eliminate 
        
        return
    
    def measure_distance(self, x1, x2_set):
        
        x2_dist = torch.subtract(x2_set, x1)
        x2_l2 = torch.linalg.norm(x2_dist.flatten(start_dim=1),ord = 2, dim=1)
        
        return x2_dist, x2_l2
    
    def ensemble_attack_order(self):
        
        num_iters = self.atk_params.iteration
        atk_order = []
        
        for t in range(num_iters):
            idx = t%len(self.advNN_idx)
            atk_order += [self.advNN_idx[idx]]
            
        self.atk_order = atk_order
    
    def sweep_victim_distance(self, og_ep, min_dist_unit_vector, ep_granularity, rep_padding, print_res = False):
        """
        Given direction of misclassification, calculate discance needed to cross boundary of victims
        """
        
        # Sweep epilson in unit direction for all NN
        num_sweep = int(np.ceil(og_ep)/ep_granularity)
        # Base NN
        base_ep = 0
        for e in range(1,num_sweep+1):
            
            delta = (min_dist_unit_vector * e * ep_granularity).unsqueeze(0)
            x_in = self.fixed_point["x"] + delta
            
            y_dist = self.models_list[self.base_nn_idx](x_in)

            y_idx = torch.argmax(y_dist, axis=1)
            
            if y_idx != self.fixed_point["y"]:
                if print_res:
                    print("og distance:", og_ep)
                    print("iteration:",e,", ep:", e * ep_granularity)
                    print("true y:", self.fixed_point["y"])
                    print("pred_y:", y_idx)
                    print("relative_y:",self.comparison_y[min_dist_idx])
                    scaled_y = self.models_list[self.base_nn_idx](self.fixed_point["x"] 
                                + (min_dist_unit_vector * num_sweep * ep_granularity).unsqueeze(0))
                    scaled_y = torch.argmax(scaled_y,axis=1)
                    print("scaled_ep_y:",scaled_y)
                base_ep = e*ep_granularity
                break
            
        # Victim NN
        victim_eps = {}
        num_sweep_v = num_sweep + rep_padding
        for v_idx in self.victim_idx:
            for e in range(1, num_sweep_v+1):
                delta = (min_dist_unit_vector * e * ep_granularity).unsqueeze(0)
                x_in = self.fixed_point["x"] + delta

                y_dist = self.models_list[v_idx](x_in)
                y_idx = torch.argmax(y_dist, axis=1)

                if y_idx != self.fixed_point["y"]:
                    victim_eps[v_idx] = e*ep_granularity
                    break
                                                                          
        return base_ep, victim_eps
    
    def legitimate_direction(self, batch_size, ep_granularity = 0.5, rep_padding = 1000, new_point = True,
                            print_res = False):
        """
        Calculate Legitimate Direction for a single point
        """
        
        # Select point of baseline comparison 
        if new_point:
            self.select_data_point()
        
        # Select set of comparison 
        self.select_comparison_set(batch_size)
        
        # Calculate X distance
        x_dists, x_dists_l2 = self.measure_distance(self.fixed_point["x"], self.comparison_set["x"])
        
        # Classify all members of comparison set
        y_pred_nn = None
        min_dist_idx = None
        min_dist_unit_vector = None
        
        # Classify each data for each classifier
        temp_classified = self.models_list[self.base_nn_idx](self.comparison_set["x"])
        y_pred_nn = torch.argmax(temp_classified,axis=1)

        # Filter twice - argmin (distance), conditioned on different label
        dist_mask = torch.where(y_pred_nn != self.fixed_point["y"], x_dists_l2, torch.max(x_dists_l2))
        min_dist_idx = torch.argmin(dist_mask)
        min_dist_unit_vector = torch.divide(x_dists[min_dist_idx], 
                                                torch.linalg.norm(x_dists[min_dist_idx].flatten(),ord=2))
        
        og_ep = torch.linalg.norm(x_dists[min_dist_idx].flatten(),ord=2).data.tolist()
            
        self.y_comparison = self.comparison_set["y"][min_dist_idx]
        
        base_ep, victim_eps = self.sweep_victim_distance(og_ep, min_dist_unit_vector, 
                                                         ep_granularity, rep_padding, print_res = False)
                                                                          
        return base_ep, victim_eps
    
    def adversarial_direction(self, ep_granularity = 0.5, rep_padding = 1000, new_point = True, 
                              print_res = False, target = True):
        """
        Calculate adversarial direction for a single point (same as before?) 
        Return base_epsilon for comparison model, and victim_eps for others
        """
        
        # Select point of baseline comparison 
        if new_point:
            self.select_data_point()
        
        # Perform a single step of attack on data point 
        x_in = self.fixed_point["x"]
        y_in = self.fixed_point["y"]
        
        if target and (self.y_comparison is not None):
            self.atk_params.target = self.y_comparison
        
        self.advNN.i_fgsm_sub(self.atk_params,x_in.unsqueeze(0),y_in.unsqueeze(0))
        
        x_adv = self.advNN.x_adv
        dist_diff = torch.subtract(x_adv, x_in)
        
        min_dist_unit_vector = torch.divide(dist_diff, torch.linalg.norm(dist_diff.flatten(),ord=2))[0]
        og_ep = torch.linalg.norm(dist_diff.flatten(),ord=2).data.tolist()
        
        base_ep, victim_eps = self.sweep_victim_distance(og_ep, min_dist_unit_vector, 
                                                         ep_granularity, rep_padding, print_res = False)
                                                                                                                                                    
        return base_ep, victim_eps
    
    def ensemble_adversarial_direction(self, ep_granularity = 0.5, rep_padding = 1000, new_point = True, 
                              print_res = False, target = True):
        
        # Select point of baseline comparison 
        if new_point:
            self.select_data_point()
        
        # Perform a single step of attack on data point 
        x_in = self.fixed_point["x"]
        y_in = self.fixed_point["y"]
        
        adv_x_in = x_in.unsqueeze(0)
        adv_y_in = y_in.unsqueeze(0)
        
        if target and (self.y_comparison is not None):
            self.atk_params.target = self.y_comparison
            
        # Decide on attack sequence
        self.ensemble_attack_order()
        
        # Alter number of iteration in params to 1 
        temp_params = copy.deepcopy(self.atk_params)
        temp_params.iteration = 1
        
        for idx in self.atk_order:
            self.advNN[idx].i_fgsm_sub(temp_params, adv_x_in, adv_y_in)
            adv_x_in = copy.deepcopy(self.advNN[idx].x_adv)
        
        # Record relevant tensors
        x_adv = copy.deepcopy(adv_x_in).cuda()
        
        dist_diff = torch.subtract(x_adv, x_in)
        
        min_dist_unit_vector = torch.divide(dist_diff, torch.linalg.norm(dist_diff.flatten(),ord=2))[0]
        og_ep = torch.linalg.norm(dist_diff.flatten(),ord=2).data.tolist()
        
        base_ep, victim_eps = self.sweep_victim_distance(og_ep, min_dist_unit_vector, 
                                                         ep_granularity, rep_padding, print_res = False)
                                                                                                                                                    
        return base_ep, victim_eps

### Legitimate Direction Calculation - FedEM
- Across test set perform classification for each of the nueral networks 
- Measure distance from a single point to all other points 
- Find the closest point with different label

In [45]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "FedEM"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_12_02_first_transfers_xadv_train_n0/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 195.91it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:30<00:00,  2.61it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [46]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]
    
weights = np.load("weights/cifar/21_12_02_first_transfers_xadv_train_n40/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

In [7]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.stack(data_y)

# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)

In [47]:
num_trials = 200
batch_size = 5000
adv_idx = [0,1]

t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
t1.base_nn_idx = 0
t1.victim_idx = [1,2,3,4]


In [48]:
dists_measure_legit = np.zeros([num_trials, len(t1.victim_idx)])
dists_measure_adv = np.zeros([num_trials, len(t1.victim_idx)])
dists_measure_adv_ensemble = np.zeros([num_trials, len(t1.victim_idx)])

for i in range(num_trials):
    print("num_trial:", i)
    t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
    t1.base_nn_idx = 0
    t1.victim_idx = [1,2,3,4]

    t1.atk_params.set_params(batch_size=1, eps=0.1, alpha=0.01, iteration = 30,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    t1.set_adv_NN(t1.base_nn_idx)

    base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = True,print_res = False)
    
    base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    t1.set_ensemble_adv_NN(adv_idx)
    base_ep_Eadv, victim_eps_Eadv = t1.ensemble_adversarial_direction(ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    idx = 0
    for key, value in victim_eps_legit.items():
        dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_adv.items():
        dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
        idx+=1
    
    idx = 0
    for key, value in victim_eps_Eadv.items():
        dists_measure_adv_ensemble[i,idx] = np.abs(base_ep_Eadv - value)
        idx+=1

num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4
num_trial: 5
num_trial: 6
num_trial: 7
num_trial: 8
num_trial: 9
num_trial: 10
num_trial: 11
num_trial: 12
num_trial: 13
num_trial: 14
num_trial: 15
num_trial: 16
num_trial: 17
num_trial: 18
num_trial: 19
num_trial: 20
num_trial: 21
num_trial: 22
num_trial: 23
num_trial: 24
num_trial: 25
num_trial: 26
num_trial: 27
num_trial: 28
num_trial: 29
num_trial: 30
num_trial: 31
num_trial: 32
num_trial: 33
num_trial: 34
num_trial: 35
num_trial: 36
num_trial: 37
num_trial: 38
num_trial: 39
num_trial: 40
num_trial: 41
num_trial: 42
num_trial: 43
num_trial: 44
num_trial: 45
num_trial: 46
num_trial: 47
num_trial: 48
num_trial: 49
num_trial: 50
num_trial: 51
num_trial: 52
num_trial: 53
num_trial: 54
num_trial: 55
num_trial: 56
num_trial: 57
num_trial: 58
num_trial: 59
num_trial: 60
num_trial: 61
num_trial: 62
num_trial: 63
num_trial: 64
num_trial: 65
num_trial: 66
num_trial: 67
num_trial: 68
num_trial: 69
num_trial: 70
num_trial: 71
nu

In [52]:
np.mean(np.average(dists_measure_legit,axis=0))

7.86375

In [50]:
np.mean(np.average(dists_measure_adv,axis=0))

14.870625

In [51]:
np.average(dists_measure_adv_ensemble,axis=0)

array([8.66, 8.28, 8.95, 8.34])

### Legitimate Direction Calculation - Local
- Across test set perform classification for each of the nueral networks 
- Measure distance from a single point to all other points 
- Find the closest point with different label

In [13]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = "local"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 3
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/21_11_06_local/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 283.02it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:27<00:00,  2.91it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.159% |Test Loss: 2.292 | Test Acc: 12.248% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [14]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]
    
weights = np.load("weights/cifar/21_09_28_first_transfers/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

In [15]:
num_trials = 200
batch_size = 5000
adv_idx = [0,1]

t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
t1.base_nn_idx = 0
t1.victim_idx = [1,2,3,4]


In [16]:
dists_measure_legit = np.zeros([num_trials, len(t1.victim_idx)])
dists_measure_adv = np.zeros([num_trials, len(t1.victim_idx)])
dists_measure_adv_ensemble = np.zeros([num_trials, len(t1.victim_idx)])

for i in range(num_trials):
    print("num_trial:", i)
    
    t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
    t1.base_nn_idx = 0
    t1.victim_idx = [1,2,3,4]

    t1.atk_params.set_params(batch_size=1, eps=0.1, alpha=0.01, iteration = 30,
                       target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x))
    t1.set_adv_NN(t1.base_nn_idx)
    
    base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = True,print_res = False)
    
    base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    t1.set_ensemble_adv_NN(adv_idx)
    base_ep_Eadv, victim_eps_Eadv = t1.ensemble_adversarial_direction(ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    idx = 0
    for key, value in victim_eps_legit.items():
        dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_adv.items():
        dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
        idx+=1
    
    idx = 0
    for key, value in victim_eps_Eadv.items():
        dists_measure_adv_ensemble[i,idx] = np.abs(base_ep_Eadv - value)
        idx+=1

num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4
num_trial: 5
num_trial: 6
num_trial: 7
num_trial: 8
num_trial: 9
num_trial: 10
num_trial: 11
num_trial: 12
num_trial: 13
num_trial: 14
num_trial: 15
num_trial: 16
num_trial: 17
num_trial: 18
num_trial: 19
num_trial: 20
num_trial: 21
num_trial: 22
num_trial: 23
num_trial: 24
num_trial: 25
num_trial: 26
num_trial: 27
num_trial: 28
num_trial: 29
num_trial: 30
num_trial: 31
num_trial: 32
num_trial: 33
num_trial: 34
num_trial: 35
num_trial: 36
num_trial: 37
num_trial: 38
num_trial: 39
num_trial: 40
num_trial: 41
num_trial: 42
num_trial: 43
num_trial: 44
num_trial: 45
num_trial: 46
num_trial: 47
num_trial: 48
num_trial: 49
num_trial: 50
num_trial: 51
num_trial: 52
num_trial: 53
num_trial: 54
num_trial: 55
num_trial: 56
num_trial: 57
num_trial: 58
num_trial: 59
num_trial: 60
num_trial: 61
num_trial: 62
num_trial: 63
num_trial: 64
num_trial: 65
num_trial: 66
num_trial: 67
num_trial: 68
num_trial: 69
num_trial: 70
num_trial: 71
nu

In [17]:
np.average(dists_measure_legit,axis=0)

array([15.37, 9.22, 16.07, 11.32])

In [18]:
np.average(dists_measure_adv,axis=0)

array([15.68, 10.15, 6.14, 6.01])

In [19]:
np.average(dists_measure_adv_ensemble,axis=0)

array([13.79, 6.75, 5.58, 5.87])